# Selenium 네이버 뉴스 댓글 스크래핑

## Selenium 및 웹 드라이버 설치

In [1]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver /usr/bin')

     |████████████████████████████████| 911kB 2.8MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.

In [2]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

## 라이브러리 import

In [3]:
import time
import pandas as pd

from selenium.common.exceptions import NoSuchElementException

## `news_scraping()`: 뉴스 스크래핑

In [22]:
def news_scraping(news_url, wd):
  press = wd.find_element_by_xpath('//*[@id="main_content"]/div[1]/div[1]/a/img').get_attribute('title')
  title = wd.find_element_by_id('articleTitle').text
  datetime = wd.find_element_by_class_name('t11').text
  article = wd.find_element_by_id('articleBodyContents').text
  article = article.replace('// flash 오류를 우회하기 위한 함수 추가','')
  article = article.replace('funtion _flash_removeCallback() {}','')
  article = article.replace('\n','')
  article = article.replace('\t','')

  good = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[1]/a/span[2]').text
  warm = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[2]/a/span[2]').text
  sad = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[3]/a/span[2]').text
  angry = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[4]/a/span[2]').text
  want = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[5]/a/span[2]').text
  recom = wd.find_element_by_xpath('//*[@id="toMainContainer"]/a/em[2]').text

  print('뉴스 :',[title,press,datetime,article,good,warm,sad,angry,want,recom,news_url])

  return [title,press,datetime,article,good,warm,sad,angry,want,recom,news_url]

## `comments_scraping()`: 뉴스 댓글 스크래핑

In [63]:
def comments_scraping(news_url, wd):

  try:
    wd.find_element_by_class_name('u_cbox_btn_view_comment').click()
    print('[댓글 더보기]',end='')
    time.sleep(1)

    while True:
      wd.find_element_by_class_name('u_cbox_btn_more').click()
      print('[더보기]',end='')
      time.sleep(1)

  except:
    pass

  btn_reply_list = wd.find_elements_by_class_name('u_cbox_btn_reply')
  for btn_reply in btn_reply_list:
    btn_reply.send_keys('\n')
    print('[답글]',end='')
    time.sleep(1)

  print('[댓글 스크래핑]')
  comments_idx = 0
  comments_df = pd.DataFrame(columns=('Contents','Name','Datetime','Recommend','Unrecommend','URL'))

  comments = wd.find_elements_by_class_name('u_cbox_comment_box')
  for comment in comments:
    try:
      name = comment.find_element_by_class_name('u_cbox_nick').text
      date = comment.find_element_by_class_name('u_cbox_date').text
      contents = comment.find_element_by_class_name('u_cbox_contents').text
      recomm = comment.find_element_by_class_name('u_cbox_cnt_recomm').text
      unrecomm = comment.find_element_by_class_name('u_cbox_cnt_unrecomm').text
      print(f' 댓글 #{comments_idx+1}:',[contents,name,date,recomm,unrecomm,news_url])
      comments_df.loc[comments_idx] = [contents,name,date,recomm,unrecomm,news_url]
      comments_idx += 1
    except NoSuchElementException as e:
      print('[삭제되거나 부적잘한 댓글]')
      continue

  return comments_df

## `scraping()`: 스크래핑 함수

In [59]:
def scraping(news_max=3):
  wd = webdriver.Chrome('chromedriver',options=chrome_options)
  wd.implicitly_wait(3)

  wd.execute_script('window.open("about:blank", "_blank");')
  tabs = wd.window_handles

  wd.switch_to.window(tabs[0])
  query = input('검색어 입력:')

  search_url = 'https://search.naver.com/search.naver?where=news&ie=utf8&sm=nws_hty&query=' + query
  wd.get(search_url)

  news_idx = 0
  news_df = pd.DataFrame(columns=('Title','Press','DateTime','Article','Good','Warm','Sad','Angry','Want','Recommend','URL'))
  comments_df = pd.DataFrame()

  while True:
    inline_list = wd.find_elements_by_class_name('txt_inline')
    for inline in inline_list:
      try:
        news_url = inline.find_element_by_class_name('_sp_each_url').get_attribute('href')
      except:
        continue

      wd.switch_to.window(tabs[1])
      wd.get(news_url)

      news_df.loc[news_idx] = news_scraping(news_url, wd)
      news_idx += 1

      df = comments_scraping(news_url, wd)
      comments_df = pd.concat([comments_df,df])

      if news_idx >= news_max:
        break
    if news_idx >= news_max:
      break

    try:
      wd.switch_to.window(tabs[0])
      wd.find_element_by_class_name('next').click()
      time.sleep(1)
    except:
      break

  wd.close()

  return news_df, comments_df

## 스크래핑 실행

In [64]:
news_df, comments_df = scraping()

검색어 입력:인공지능
뉴스 : ['"증강현실(AR)과 인공지능(AI)가 만나 \'공간 스타일\' 추천해요"', '한국경제', '2020.08.13. 오전 9:07', "어반베이스, 홈인테리어 AR앱에 스페이스AI 기능 추가미리 공간에 배치할 수 있어공간데이터 플랫폼 어반베이스(대표 하진우)는 자사의 '홈인테리어 증강현실(AR) 앱'에 공간분석 인공지능(스페이스 AI)을 더해 새롭게 개편했다고 13일 밝혔다. AR와 AI가 만나면서 고객이 선호하는 공간 스타일에 맞는 제품을 추천하고 추천 제품을 실제 공간에 배치해보는 일련의 과정이 하나의 앱에서 가능하게 됐다.어반베이스는 2018년 1월 증강현실로 집을 꾸미는 ‘어반베이스 AR’ 앱 출시 후 앱의 지속적인 활용성을 높일 수 있는 방안을 고민해 왔다. 증강현실의 핵심 기능인 ‘배치’에만 초점을 맞춰서는 이벤트 요소가 강한 일회성 앱에 머물 수밖에 없다고 판단했다. 여기에 인공지능 큐레이션을 추가해 사용자 개개인의 공간에 특화된 증강현실 앱으로 발전시키기 위해 노력했다.어반베이스의 공간분석 인공지능인 ‘스페이스 AI’는 실내공간 이미지를 분석해 공간 정보를 제공하는 기술이다. 이미지 내 공간을 거실·방·주방·욕실 등 4가지 유형으로 구분하고, 공간에 위치한 90여 종의 사물을 인식한다. 이렇게 검출된 공간 유형과 사물 데이터를 기반으로 인테리어 스타일을 분석한 후 해당 스타일에 어울리는 제품을 추천해준다.스페이스 AI의 API(개방형 응용프로그램개발환경)를 활용하면 앱은 물론 웹 서비스에도 공간분석 기능을 추가할 수 있다. 스페이스 AI는 현재 SK그룹의 주요 ICT(정보통신기술) 관계사가 참여하는 ‘SK 오픈 API 포털’에 외부업체 최초로 등록돼 있다. 최근에는 국내 특허출원을 마쳤다.어반베이스 AR 이용자들은 디바이스 내 저장된 이미지 또는 스냅샷을 찍어 스페이스 AI를 활용할 수 있다. 앱 화면의 오른쪽 상단에 있는 ‘스페이스 AI’ 버튼을 누르고 공간 분석을 원하는 이미지를 업로드하면 해당 공간에 어울리는

In [65]:
news_df

,Title,Press,DateTime,Article,Good,Warm,Sad,Angry,Want,Recommend,URL
0,"""증강현실(AR)과 인공지능(AI)가 만나 '공간 스타일' 추천해요""",한국경제,2020.08.13. 오전 9:07,"어반베이스, 홈인테리어 AR앱에 스페이스AI 기능 추가미리 공간에 배치할 수 있어공...",0,0,0,0,0,,https://news.naver.com/main/read.nhn?mode=LSD&...
1,인공지능(AI)으로 초등학교 수업을?,KBS,2020.08.12. 오전 7:01,이제 수그러지나 싶던 코로나19 신규 확진자 수가 최근 연일 30명 안팎을 기록하고...,2,0,1,10,0,,https://news.naver.com/main/read.nhn?mode=LSD&...
2,"삼성전자 해외연구소, 인공지능 번역 경진대회서 잇단 우승",머니투데이,2020.08.14. 오후 1:05,[머니투데이 심재현 기자]IWSLT 경진대회에 참가한 삼성전자 폴란드 연구소 팀. ...,1,0,0,0,0,,https://news.naver.com/main/read.nhn?mode=LSD&...


In [66]:
comments_df

,Contents,Name,Datetime,Recommend,Unrecommend,URL
0,전학생의 독감 예방접종 찬성합니다.코로나 아니더라도 독감 유행일 때마다 한 명이 걸...,shin****,2020.08.11. 23:32,6,3,https://news.naver.com/main/read.nhn?mode=LSD&...
1,웅진씽크빅이랑 대교 눈높이 잘되었습니다. 교육부에서 웅진꺼 이용할거 같은데우리 아이...,keli****,2020.08.12. 02:31,0,0,https://news.naver.com/main/read.nhn?mode=LSD&...
2,그거 가지고 감염예방이 되겠습니까?,redg****,2020.08.11. 23:41,0,0,https://news.naver.com/main/read.nhn?mode=LSD&...
3,이런 돈으로 교과서을1학년 수준으로맞추어라.아니면유치원부터공부하라고해라.1학년교과서...,kas4****,2020.08.12. 12:51,0,3,https://news.naver.com/main/read.nhn?mode=LSD&...
